In [4]:
from qat.lang.AQASM import *
from qat.lang.AQASM.qftarith import QFT
from qat.qpus import PyLinalg
from qat.qpus import get_default_qpu
import numpy as np
import matplotlib.pyplot as plt



# Algoritmo de Shor

Neste notebook, vamos implementar um algoritmo quântico para fatorar o número 15. Escreveremos um código de implementação do Algoritmo de Shor em `myQLM`, seguindo o material apresentado em sala.

Material de apoio:
- **Transformada de Fourier Quântica**: aula qft
- **Estimativa de Fase Quântica**: aula qpe
- **Algoritmo de Shor**: aula shor


## Revisão do Algoritmo de Shor
Lembrando da aula X, O Algoritmo de Shor pode ser utilizado para encontrar fatores de um número inteiro, $N$, sendo

$$N = pq$$

onde $p$ e $q$ são números primos. Isso é feito em quatro etapas principais, semelhantes à implementação da estimativa de fase quântica. Neste caso, usaremos dois registradores de qubits. O primeiro registro terá $n = \lceil \log_2 N^2 \rceil = \lceil 2 \log_2 N \rceil$ qubits e conterá os qubits de medição. O segundo registrador terá $m = \lceil \log_2 N \rceil$ qubits, e será o autoestado para a estimativa da fase quântica. Para o número $15$, podemos usar $m=n$, visto que a ordem é sempre uma potência de $2$.

1. Primeiro, começamos inicializando nossos qubits. Criamos uma superposição de todos os estados de base computacional $2^n$ nos $n$ qubits do primeiro registrador, aplicando uma porta Hadamard ($H$) em cada qubit começando no estado $\vert0\rangle^{\otimes n}$. Também inicializamos os qubits de destino $m = \lceil \log_2 N \rceil$  no estado $\vert1\rangle$. Aqui, o expoente $\otimes n$ significa que temos um produto tensorial dos estados de $n = m$ qubits.

2. Em segundo lugar, aplicamos o operador unitário $U_a|x\rangle = |ax \mod N\rangle $, que será usado para realizar a exponenciação modular no segundo registrador, controlando-o com cada um dos diferentes qubits do primeiro registrador. O esquema abaixo mostra a ordenação e as respectivas potências.

$$$$

<img src="imgs/shorimage.png" width=800 />

$$$$

3. Em terceiro lugar, aplicamos uma transformada de Fourier quântica inversa nos qubits do primeiro registrador.

4. Finalmente, medimos os primeiros $n$ qubits, colapsando para o estado $|l\rangle$ .

Depois que os resultados da medição forem determinados, precisaremos fazer um pós-processamento clássico adicional para determinar os fatores ou decidir executar o programa novamente.

5. Calculamos $\frac{l}{2^n}$ e tomamos o denominador como candidato a $r$.

6. Testar se $r$ é período. Se $ a^r = 1 \mod N$, então $r$ é o período e é garantido que os fatores primos de $N$ serão obtidos através da relação

$$p,q = \text{mdc}(a^{r/2} \pm 1, N)$$






## Implementando o Algoritmo de Shor com o `myQLM`

Neste notebook, vamos implementar o Algoritmo de Shor para fatorar o número 15.


### 1. Inicializando os qubits

Inicializamos o registrador de medida como uma superposição uniforme através da operação $H^{\otimes n}|0\rangle^{\otimes n}$. O segundo registrador, onde será aplicada a exponenciação modular, é inicializado no estado $|1\rangle \equiv |00...001\rangle$.

Logo, o estado inicializado, $|\psi_0\rangle$, é dado por

$$|\psi_0\rangle = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1}|i\rangle\otimes |1\rangle$$

#### 1.1 Código de inicialização

In [5]:
n=4; m=4; num_qubits = n+m
a=7; N=15

def initialize_qubits():

    init_circ = QRoutine()
    wires = init_circ.new_wires(num_qubits)

    for i in range(n):
        init_circ.apply(H, wires[i])

    init_circ.apply(X, wires[num_qubits - 1])   

    return init_circ

In [6]:
circuit = Program()
qubits = circuit.qalloc(num_qubits)
circuit.apply(initialize_qubits(), qubits)
circuit = circuit.to_circ()
%qatdisplay circuit --svg

### 2. Exponenciação Modular Quântica

Nessa etapa, precisamos gerar os valores da exponenciação modular $7^{2^i} \mod 15$ no segundo registrador, e para cada inteiro $i$ será associado um qubit no primeiro registrador que irá controlar esta porta.

Para tal, precisamos implementar várias vezes o operador:

$U_7|y\rangle = |7 y\mod 15\rangle$

Que é descrito em https://arxiv.org/pdf/1202.6614.pdf pelo circuito:

$$$$

<img src="imgs/M7.jpg" width=200 />

$$$$

Para cada $i$, basta aplicarmos este circuito $2^i$ vezes de forma controlada a partir do valor inicial 1.

O circuito acima pode ser controlado estendendo as portas $X$ para CNOTS com um qubit de controle e abrindo a porta SWAP em sua decomposição por CNOTs. Assim, cada CNOT pode ser controlado o estendendo para uma porta Toffoli (CCNOT) com o qubit de controle. Vide imagens abaixo:

$$$$

<img src="imgs/M7Aberto.jpg" width=400 />

$$$$

$$$$

<img src="imgs/CM7.jpg" width=400 />

$$$$

Para generalizar esta ideia com um $a$ diferente de 7, basta obter a representação do $U_a$ em CNOTS e SWAPS e convertê-lo para uma versão controlada. O material de referência mostra circuitos para outros valores de $a$


In [7]:
def controlled_modular_exponentiation(i):
    circ = QRoutine()
    wires = circ.new_wires(1 + 4) #1 qubit de controle e 4 onde ocorrem a multiplicação
    for j in range(2**i):
    #Implementando C(M_7):
        circ.apply(CNOT,0,1)
        circ.apply(CNOT,0,2)
        circ.apply(CNOT,0,3)
        circ.apply(CNOT,0,4)

        #SWAP 1
        circ.apply(CCNOT,0,2,3)
        circ.apply(CCNOT,0,3,2)
        circ.apply(CCNOT,0,2,3)

        #SWAP 2
        circ.apply(CCNOT,0,1,2)
        circ.apply(CCNOT,0,2,1)
        circ.apply(CCNOT,0,1,2)

        #SWAP 3
        circ.apply(CCNOT,0,1,4)
        circ.apply(CCNOT,0,4,1)
        circ.apply(CCNOT,0,1,4)

    return circ
            

### 3. Criando o programa

O programa consiste na aplicação das três principais operações descritas acima:

1. Inicialização do circuito
2. Exponenciação modular
3. Transformada Quântica de Fourier (QFT)

As medidas serão feitas no registrador da QFT e usadas no pós processamento a fim de se encontrar o período $r$.


In [8]:
shor = Program()
qr = shor.qalloc(num_qubits)

init_circ = initialize_qubits()

shor.apply(init_circ, qr)

for i in range(4):
    modularExponentiation = controlled_modular_exponentiation(i)
    shor.apply(modularExponentiation,qr[3-i],qr[4],qr[5],qr[6],qr[7])


QFT(4)(qr[0], qr[1], qr[2], qr[3])

circuit = shor.to_circ()

#Obs: na interface web do jupyter é possível dar zoom na imagem e visualizar melhor o circuito
%qatdisplay circuit --svg

In [9]:
job = circuit.to_job()
result = get_default_qpu().submit(job)

states = []
for sample in result:
     print("State %s: probability %s " % (sample.state, sample.probability))
     states.append(sample.state)

State |00000001>: probability 0.06249999999999992 
State |00000100>: probability 0.06249999999999992 
State |00000111>: probability 0.06249999999999992 
State |00001101>: probability 0.06249999999999992 
State |00010001>: probability 0.06249999999999992 
State |00010100>: probability 0.06249999999999992 
State |00010111>: probability 0.06249999999999992 
State |00011101>: probability 0.06249999999999992 
State |00100001>: probability 0.06249999999999992 
State |00100100>: probability 0.06249999999999992 
State |00100111>: probability 0.06249999999999992 
State |00101101>: probability 0.06249999999999992 
State |00110001>: probability 0.06249999999999992 
State |00110100>: probability 0.06249999999999992 
State |00110111>: probability 0.06249999999999992 
State |00111101>: probability 0.06249999999999992 


In [10]:
# Converte os autoestados medidos no registrador da QFT em decimais
import string
decimals = [[str(int(b)) for b in states[i]] for i in range(len(states))]

for i,lista in enumerate(decimals):
    decimals[i] = list(reversed(lista[:n]))
    decimals[i] = int(''.join(decimals[i]),2)
    

decimals = set(decimals)


### Pós processamento dos valores medidos

Como visto na teoria, para encontrar o valor do período $r$, usando $a = 7$, para $N=15$, tomaremos o denominador de $l/2^n$ como um candidato a $r$. Como vimos, $l$ é a representação decimal de um resultado obtido através de uma medida no primeiro registrador e $n=4$.

In [11]:
from math import gcd, modf

# funcao que retorna o denominador de l/2^n
def fraction(num, denom):
    if denom==0:
        raise ZeroDivisionError("denominador zero")
    d = gcd(num, denom)
    num = num//d
    denom = denom//d
    return (num, denom)

# Encontrando os fatores primos de N
prime_factors = (); success = []; total_try = []

for decimal in decimals:

    total_try.append(0)
    # calcula o candidato a r
    print("decimal medido: ", decimal)
    numerator, r = fraction(decimal, 2**n)
    print("candidato a r: ", r, "\n ")

    # verifica se r é par
    if r % 2 != 0 or r == 0:
        continue
        
    # calcula a^r mod N e verifica se r é periodo
    a_r_modulo_N = np.mod(a**r, N)
    if a_r_modulo_N == 1:

        success.append(0)    
        guesses = gcd(a**int(r/2) + 1, N), gcd(a**int(r/2) - 1, N)
        prime_factors = guesses[0], guesses[1]
        

print("\n Fatores primos encontrados: ", prime_factors)

print("Probabilidade de sucesso: ", 100*len(success)/len(total_try), "% ")

decimal medido:  0
candidato a r:  1 
 
decimal medido:  8
candidato a r:  2 
 
decimal medido:  4
candidato a r:  4 
 
decimal medido:  12
candidato a r:  4 
 

 Fatores primos encontrados:  (5, 3)
Probabilidade de sucesso:  50.0 % 
